In [7]:
import os
import json
from pathlib import Path

import pandas as pd

from neuralhydrology.nh_run import eval_run

In [8]:
# load allowed GPU ids
f = open("gpu.json")
gpus = json.load(f)
f.close()

In [9]:
epoch = 19
epoch_str = str(epoch) if epoch / 100 > 1 else f"0{epoch}"

In [10]:
run_dir = Path(f"./runs/{sorted(os.listdir('./runs'))[-1]}")

In [5]:
def show_eval_results(period: str, epoch: str = "010") -> pd.DataFrame:
    df = pd.read_csv(run_dir / f"{period}" / f"model_epoch{epoch}" / f"{period}_metrics.csv", dtype={'basin': str})
    df = df.set_index('basin')

    # Compute the median NSE from all basins, where discharge observations are available for that period
    print(f"Median NSE of the {period} period {df['NSE'].median():.3f}")
    print(f"Mean NSE of the {period} period {df['NSE'].mean():.3f}")
    print(df)
    
    return df

In [6]:
eval_run(run_dir, period="train", epoch=epoch, gpu=gpus[0])
df_train = show_eval_results("train", epoch=epoch_str)

RuntimeError: This machine does not have GPU #5 

In [ ]:
eval_run(run_dir, period="validation", epoch=epoch, gpu=gpus[0])
df_val = show_eval_results("validation", epoch=epoch_str)

# Validation:  10%|▉         | 4/42 [00:00<00:09,  4.12it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Validation:  45%|████▌     | 19/42 [00:04<00:05,  4.11it/s]

The following basins had not enough valid target values to calculate a standard deviation: 13035. NSE loss values for this basin will be NaN.


# Validation:  67%|██████▋   | 28/42 [00:07<00:03,  4.03it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Validation: 100%|██████████| 42/42 [00:10<00:00,  3.96it/s]
Median NSE of the validation period 0.352
Mean NSE of the validation period 0.361
            NSE
basin          
11001  0.585831
11068 -0.384398
11126  0.854831
11129  0.764054
11163       NaN
11164  0.628861
11275  0.008243
11293  0.445322
11397  0.074017
11433  0.163400
11469  0.737924
12002  0.328793
12008 -0.214038
12032 -0.014399
12072  0.308092
12075  0.554264
12564  0.351611
13002  0.343412
13005  0.310955
13035       NaN
13038  0.538745
13048  0.325846
13064  0.195148
13090  0.760546
13091  0.494310
13115  0.540329
13128  0.364835
13221  0.357655
19013       NaN
19022  0.079550
19195  0.224465
19196  0.485065
19205  0.395473
19208  0.168145
19218  0.329480
19243  0.229327
19289  0.247906
19300  0.547841
19302  0.711471
19462  0.466795
19463  0.461266
77819  0.309240


In [ ]:
eval_run(run_dir, period="test", epoch=epoch, gpu=gpus[0])
df_test = show_eval_results("test", epoch=epoch_str)

# Evaluation:  10%|▉         | 4/42 [00:00<00:09,  4.12it/s]

The following basins had not enough valid target values to calculate a standard deviation: 11163. NSE loss values for this basin will be NaN.


# Evaluation:  45%|████▌     | 19/42 [00:04<00:05,  4.03it/s]

The following basins had not enough valid target values to calculate a standard deviation: 13035. NSE loss values for this basin will be NaN.


# Evaluation:  67%|██████▋   | 28/42 [00:07<00:03,  4.07it/s]

The following basins had not enough valid target values to calculate a standard deviation: 19013. NSE loss values for this basin will be NaN.


# Evaluation: 100%|██████████| 42/42 [00:10<00:00,  3.95it/s]
Median NSE of the test period -0.000
Mean NSE of the test period -56.034
               NSE
basin             
11001     0.681770
11068    -1.040793
11126     0.783455
11129     0.742770
11163          NaN
11164     0.728758
11275     0.031301
11293     0.525794
11397     0.060387
11433     0.403923
11469     0.828736
12002    -0.769617
12008    -5.358892
12032    -0.111583
12072    -0.000325
12075    -0.820714
12564     0.044145
13002    -0.410889
13005     0.326431
13035          NaN
13038 -1796.698364
13048     0.102794
13064     0.276326
13090     0.427330
13091     0.398711
13115     0.592826
13128     0.210627
13221     0.110321
19013          NaN
19022    -7.104419
19195    -1.616225
19196   -10.348888
19205    -1.253524
19208    -1.192095
19218    -8.354615
19243   -59.410980
19289   -20.666443
19300    -1.938063
19302    -5.080658
19462   -11.104247
19463  -259.757690
77819     0.422217


In [ ]:
df_train = df_train.rename(columns={"NSE": "NSE_train", "KGE": "KGE_train"})
df_val = df_val.rename(columns={"NSE": "NSE_val", "KGE": "KGE_val"})
df_test = df_test.rename(columns={"NSE": "NSE_test", "KGE": "KGE_test"})

In [ ]:
df_tmp = pd.merge(df_train, df_val, left_index=True, right_index=True)
df = pd.merge(df_tmp, df_test, left_index=True, right_index=True)
df.head()

File will be saved in the current runs directory

In [ ]:
df.to_csv(f"{str(run_dir)}/eval.csv")